In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [21]:
result1 = pd.read_csv('Ensamble XGB-RF-KNN new values.csv').rename(columns={"damage_grade":"damage_grade_1"})
result2 = pd.read_csv('submission_27_7_E_6.csv').rename(columns={"damage_grade":"damage_grade_2"})
result3 = pd.read_csv('Ensamble XGB-RF-KNN-f.csv').rename(columns={"damage_grade":"damage_grade_3"})
result1

,building_id,damage_grade_1
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3
...,...,...
86863,310028,2
86864,663567,2
86865,1049160,2
86866,442785,2


In [39]:
merged = result1.merge(result2,on="building_id").merge(result3,on="building_id")

# merged["most common"] = merged.apply(lambda x: [x[damage_grade_1]])



merged["all_equals"] = merged.apply(lambda x: 1 if (x["damage_grade_1"] == x["damage_grade_2"] == x["damage_grade_3"]) else 0, axis = 1)

In [42]:
merged.mean()


building_id       526627.878851
damage_grade_1         2.208800
damage_grade_2         2.205231
damage_grade_3         2.210181
all_equals             0.918923
dtype: float64

In [68]:
def most_common(x):
    most = x["damage_grade_1"]
    if x["damage_grade_2"] == most :
        return most
    if x["damage_grade_3"] == most :
        return most
    if x["damage_grade_3"] == x["damage_grade_2"] :
        return x["damage_grade_3"]
    return x["damage_grade_2"]
    
    
    
merged["most"] = merged.apply(most_common, axis = 1)
    
final = merged[["building_id", "most"]]
final = final.rename(columns={"most":"damage_grade"})


In [69]:
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
my_submission = pd.DataFrame(data=final["damage_grade"].to_numpy(), columns=submission_format.columns, index=submission_format.index)
my_submission.to_csv("merged_results.csv")